In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from itertools import combinations
from scipy.stats import mannwhitneyu
import warnings

import matplotlib.colors as mcolors
from matplotlib.colors import Normalize
from matplotlib.patches import FancyArrowPatch
import matplotlib.lines as mlines
from scipy.stats import gaussian_kde
from scipy.signal import savgol_filter
from scipy.stats import mannwhitneyu
from peregrin.scripts import PlotParams
import seaborn as sns
from itertools import combinations
import altair as alt
import plotly.graph_objects as go





In [2]:
warnings.simplefilter("ignore", FutureWarning)

In [18]:
spot_df_intconds = pd.read_csv(r"C:\Users\modri\Desktop\python\Peregrin\Peregrin\test data\Spot stats 2025-09-07 conds-int.csv")
spot_df_strconds = pd.read_csv(r"C:\Users\modri\Desktop\python\Peregrin\Peregrin\test data\Spot stats 2025-09-07 conds-str.csv")
track_df_intconds = pd.read_csv(r"C:\Users\modri\Desktop\python\Peregrin\Peregrin\test data\Track stats 2025-09-07 conds-int.csv")
track_df_strconds = pd.read_csv(r"C:\Users\modri\Desktop\python\Peregrin\Peregrin\test data\Track stats 2025-09-07 conds-str.csv")

In [19]:
def _generate_random_color():
    """
    Generate a random color in hexadecimal format.

    """

    r = np.random.randint(0, 255)   # Red LED intensity
    g = np.random.randint(0, 255)   # Green LED intensity
    b = np.random.randint(0, 255)   # Blue LED intensity

    return '#{:02x}{:02x}{:02x}'.format(r, g, b)

def _generate_random_grey():
    """
    Generate a random grey color in hexadecimal format.

    """

    n = np.random.randint(0, 240)  # All LED intensities

    return '#{:02x}{:02x}{:02x}'.format(n, n, n)

def _make_cmap(elements, cmap):
    """
    Generate a qualitative colormap for a given list of elements.

    """

    n = len(elements)   # Number of elements in the dictionary
    if n == 0:          # Return an empty list if there are no elements
        return []       
    
    cmap = plt.get_cmap(cmap)                                   # Get the colormap
    colors = [mcolors.to_hex(cmap(i / n)) for i in range(n)]    # Generate a color for each element

    return colors

def _get_cmap(c_mode):
    """
    Get a colormap according to the selected color mode.

    """

    if c_mode == 'greyscale LUT':
        return plt.cm.gist_yarg
    elif c_mode == 'jet LUT':
        return plt.cm.jet
    elif c_mode == 'brg LUT':
        return plt.cm.brg
    elif c_mode == 'hot LUT':
        return plt.cm.hot
    elif c_mode == 'gnuplot LUT':
        return plt.cm.gnuplot
    elif c_mode == 'viridis LUT':
        return plt.cm.viridis
    elif c_mode == 'rainbow LUT':
        return plt.cm.rainbow
    elif c_mode == 'turbo LUT':
        return plt.cm.turbo
    elif c_mode == 'nipy_spectral LUT':
        return plt.cm.nipy_spectral
    elif c_mode == 'gist_ncar LUT':
        return plt.cm.gist_ncar
    else:
        return None


In [30]:
def index_uid(df):
    grp = df.groupby(['Condition', 'Replicate', 'Track ID'], sort=False)

    # ---- Add unique per-track index (1-based) ----
    df['Track UID'] = grp.ngroup()
    df.set_index(['Track UID'], drop=True, append=False, inplace=True, verify_integrity=False)

Spot_df =index_uid(spot_df_intconds)

In [ ]:
def Visualize_tracks_matplotlib(
        Spots_df: pd.DataFrame, 
        # Tracks_df: pd.DataFrame, 
        *args,
        condition: None = None, 
        replicate: None = None,
        c_mode: str = 'random colors', 
        only_one_color: str = 'blue', 
        lut_scaling_metric: str = 'Track displacement', 
        background: str = 'dark', 
        smoothing_index: float = 0.0, 
        lw: float = 1.0, 
        show_tracks: bool = True, 
        grid: bool = True, 
        arrows: bool = False, 
        arrowsize: int = 1
        ):

    if show_tracks:
        pass
    else:
        lw=0

    Spots = Spots_df.copy()


    if condition == None or replicate == None:
        pass
    else:
        try:
            condition = int(condition)
        except ValueError or TypeError:
            pass
        try:
            replicate = int(replicate)
        except ValueError or TypeError:
            pass


    sort_cols = ['Condition', 'Replicate', 'Track ID', 'Time point']
    missing_cols = [col for col in sort_cols if col not in Spots.columns]
    if not missing_cols:
        if condition == 'all':
            Spots = Spots.sort_values(by=sort_cols)
        elif condition != 'all' and replicate == 'all':
            Spots = Spots[Spots['Condition'] == condition].sort_values(by=sort_cols)
        elif condition != 'all' and replicate != 'all':
            Spots = Spots[(Spots['Condition'] == condition) & (Spots['Replicate'] == replicate)].sort_values(by=sort_cols)
    else:
        return plt.gcf()


    if background =='light':
        grid_color = 'gainsboro'
        face_color = 'white'
        grid_alpha = 0.5
        if grid:
            grid_lines = '-.'
        else:
            grid_lines = 'None'
    else:
        grid_color = 'silver'
        face_color = 'darkgrey'
        grid_alpha = 0.75
        if grid:
            grid_lines = '-.'
        else:
            grid_lines = 'None'
    

    np.random.seed(42)  # For reproducibility

    if c_mode in ['random colors', 'random greys', 'only-one-color']:
        colormap = None

        if c_mode in ['random colors']:
            track_colors = [_generate_random_color() for _ in range(len(Spots.index.unique()))]
        elif c_mode in ['random greys']:
            track_colors = [_generate_random_grey() for _ in range(len(Spots.index.unique()))]
        else:
            track_colors = [only_one_color for _ in range(len(Spots.index.unique()))]
        
        Spots['Track color'] = Spots.index.map(dict(zip(Spots.index.unique(), track_colors)))

    elif c_mode in ['differentiate conditions', 'differentiate replicates']:

        if c_mode == 'differentiate conditions':
            colormap = plt.get_cmap('Set1')  # Use qualitative colormap
            unique_vals = Spots_df['Condition'].unique()
            val_column = 'Condition'
        else:
            colormap = plt.get_cmap('Set1')
            unique_vals = Spots_df['Replicate'].unique()
            val_column = 'Replicate'

        # Assign colors to each unique category
        val_to_color = {
            val: colormap(i % colormap.N)  # Wrap around if more values than colors
            for i, val in enumerate(sorted(unique_vals))
        }
        # Map those colors to the tracks
        Tracks_df['Color'] = Tracks_df[val_column].map(val_to_color)

    else:
        
        colormap = _get_cmap(c_mode)
        metric_min = Spots_df[lut_scaling_metric].min()
        metric_max = Spots_df[lut_scaling_metric].max()


    fig, ax = plt.subplots(figsize=(13, 10))

    x_min = Spots_df['X coordinate'].min()
    x_max = Spots_df['X coordinate'].max()
    y_min = Spots_df['Y coordinate'].min()
    y_max = Spots_df['Y coordinate'].max()

    ax.set_aspect('1', adjustable='box')
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_xlabel('X coordinate [microns]')
    ax.set_ylabel('Y coordinate [microns]')
    ax.set_title('Track Visualization', fontsize=12)
    ax.set_facecolor(face_color)
    ax.grid(True, which='both', axis='both', color=grid_color, linestyle=grid_lines, linewidth=1, alpha=grid_alpha)

    # Manually set the major tick locations and labels
    x_ticks_major = np.arange(x_min, x_max, 200)  # Adjust the step size as needed
    y_ticks_major = np.arange(y_min, y_max, 200)  # Adjust the step size as needed
    ax.set_xticks(x_ticks_major)
    ax.set_yticks(y_ticks_major)
    ax.set_xticklabels([f'{tick:.0f}' for tick in x_ticks_major])
    ax.set_yticklabels([f'{tick:.0f}' for tick in y_ticks_major])

    # Enable minor ticks and set their locations
    ax.minorticks_on()
    x_ticks_minor = np.arange(x_min, x_max, 50)  # Minor ticks every 50 microns
    y_ticks_minor = np.arange(y_min, y_max, 50)  # Minor ticks every 50 microns
    ax.set_xticks(x_ticks_minor, minor=True)
    ax.set_yticks(y_ticks_minor, minor=True)
    ax.tick_params(axis='both', which='major', labelsize=8)


    # Spots_grouped = Spots_df.groupby(['Condition', 'Replicate', 'Track ID'])
    # Tracks_df.set_index(['Condition', 'Replicate', 'Track ID'], inplace=True)

    fig = go.Figure()

    for idx in Spots_df.index:
        """
        - group_keys is a tuple like: ('Condition_A', 'Rep1', 'Track_001')
        - group_df is the actual dataframe for that group
        
        """

        track_row = Tracks_df.loc[idx]
        track_row['Condition'] = cond
        track_row['Replicate'] = repl
        track_row['Track ID'] = track


        if colormap is not None and c_mode in ['differentiate conditions', 'differentiate replicates']:
            key = track_row[val_column]  # val_column is either 'Condition' or 'Replicate'
            color = colormap(unique_vals.tolist().index(key) % colormap.N)  # consistent mapping
            group_df['Color'] = mcolors.to_hex(color)
            
        elif colormap is not None:
            # This is for metric-based color mapping (quantitative)
            norm = plt.Normalize(metric_min, metric_max)
            color = colormap(norm(track_row[lut_scaling_metric]))
            group_df['Color'] = mcolors.to_hex(color)

        elif c_mode in ['random colors', 'random greys']:
            group_df['Color'] = track_row['Color']

        elif c_mode == 'only-one-color':
            group_df['Color'] = only_one_color


        if (type(smoothing_index) is int or type(smoothing_index) is float) and smoothing_index > 1:
            group_df['X coordinate'] = group_df['X coordinate'].rolling(window=smoothing_index, min_periods=1).mean()
            group_df['Y coordinate'] = group_df['Y coordinate'].rolling(window=smoothing_index, min_periods=1).mean()
        else:
            group_df['X coordinate'] = group_df['X coordinate']
            group_df['Y coordinate'] = group_df['Y coordinate']

        ax.plot(group_df['X coordinate'], group_df['Y coordinate'], color=group_df['Color'].iloc[0], linewidth=lw)


        if len(group_df['X coordinate']) > 1 & arrows:
            # Use trigonometry to calculate the direction (dx, dy) from the angle
            dx = np.cos(track_row['Direction mean (rad)'])  # Change in x based on angle
            dy = np.sin(track_row['Direction mean (rad)'])  # Change in y based on angle

            # Create an arrow to indicate direction
            arrow = FancyArrowPatch(
                posA=(group_df['X coordinate'].iloc[-2], group_df['Y coordinate'].iloc[-2]),  # Start position (second-to-last point)
                posB=(group_df['X coordinate'].iloc[-2] + dx, group_df['Y coordinate'].iloc[-2] + dy),  # End position based on direction
                arrowstyle='-|>',  # Style of the arrow (you can adjust the style as needed)
                color=group_df['Color'].iloc[0],  # Set the color of the arrow
                mutation_scale=arrowsize,  # Scale the size of the arrow head (adjust this based on the plot scale)
                linewidth=1.2,  # Line width for the arrow
                zorder=30  # Ensure the arrow is drawn on top of the line
            )

            # Add the arrow to your plot (if you're using a `matplotlib` figure/axes)
            plt.gca().add_patch(arrow)


    # Tracks_df.reset_index(drop=False, inplace=True)

    return plt.gcf()


In [34]:
Visualize_tracks_matplotlib(
    Spots_df=spot_df_intconds, 
    condition=1,
    replicate='all',
)

           Track ID  Time point  X coordinate  Y coordinate  Condition  \
Track UID                                                                
865           395.0      9720.0          0.00        640.25          1   
865           395.0      9840.0          0.00        636.35          1   
865           395.0      9960.0          0.00        634.40          1   
865           395.0     10080.0          0.00        630.50          1   
865           395.0     10200.0          0.00        626.60          1   
865           395.0     10320.0          0.00        624.00          1   
865           395.0     10440.0          0.00        620.75          1   
865           395.0     10560.0          0.00        617.50          1   
866           396.0      9840.0        664.95        391.30          1   
866           396.0      9960.0        664.95        394.55          1   
866           396.0     10080.0        661.70        391.95          1   
866           396.0     10200.0       

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>